In [52]:
# Import dependencies
import pandas as pd
import numpy as np
import yfinance as yf

### Value At Risk (VaR)

VaR is a statistical estimate that tells us the amount of loss we will not exceed over a certain period of time, at a confidence level. VaR assumes the distribution of return is Normal.

VaR can be calculated with the following steps:
1) Calculate periodic returns of the stocks in the portfolio
<br>
2) Create a covariance matrix based on the returns
<br>
3) Calculate the portfolio mean and standard deviation
(weighted based on investment levels of each stock in portfolio)
<br>
4) Calculate the inverse of the normal cumulative distribution (PPF) with a specified confidence interval, standard deviation, and mean
<br>
5) Estimate the value at risk (VaR) for the portfolio by subtracting the initial investment from the calculation in step (4)

In [228]:
# Get returns of the portfolio
tickers = ['AAPL', 'GOOG', 'TSLA', 'NVDA']
returns = {}
for t in tickers:
    df = yf.Ticker(t)
    df = df.history(period="max")

    df['return'] = df['Close'].pct_change(1)
    returns[t] = np.log1p(df['return'])

returns = pd.DataFrame(returns)
returns.dropna(inplace=True)
returns

,AAPL,GOOG,TSLA,NVDA
Date,,,,
2010-06-30,-0.018279,-0.020708,-0.002515,-0.026101
2010-07-01,-0.012200,-0.012347,-0.081723,0.016513
2010-07-02,-0.006217,-0.006712,-0.134312,-0.012604
2010-07-06,0.006820,-0.001100,-0.175470,-0.010789
2010-07-07,0.039587,0.031889,-0.019430,0.047192
...,...,...,...,...
2022-04-11,-0.025847,-0.031950,-0.049535,-0.053392
2022-04-12,0.011458,-0.011016,0.011229,-0.019024
2022-04-13,0.016210,0.014780,0.035259,0.031988


In [229]:
# Choosing historical period
start = '2017-04-19'
end = '2022-04-19'
period = pd.to_datetime(end) - pd.to_datetime(start)

ret = returns[(returns.index > start) & (returns.index < end)]
ret

,AAPL,GOOG,TSLA,NVDA
Date,,,,
2017-04-20,0.012433,0.004096,-0.009901,0.015726
2017-04-21,-0.001194,0.001828,0.010163,0.004139
2017-04-24,0.009583,0.022944,0.007920,0.012413
2017-04-25,0.006177,0.010997,0.018527,0.017238
2017-04-26,-0.005899,-0.000654,-0.011603,-0.006898
...,...,...,...,...
2022-04-11,-0.025847,-0.031950,-0.049535,-0.053392
2022-04-12,0.011458,-0.011016,0.011229,-0.019024
2022-04-13,0.016210,0.014780,0.035259,0.031988


In [247]:
from scipy.stats import norm, t

class ValueAtRisk:
    def __init__(self, initial_investment, port_weights, ret, conf_level=.05):
        assert len(port_weights) == len(ret.columns)
        self.initial_investment = initial_investment
        self.port_weights = np.array(port_weights)
        self.ret = ret
        self.conf_level = conf_level
    
        # Calculate the covariance matrix
        cov_matrix = self.ret.cov()

        # Calculate the portfolio mean
        ret_mean = self.ret.mean()

        # Calculate the portfolio mean
        port_mean = ret_mean.dot(self.port_weights)

        # Calculate portfolio standard deviation
        # (w ** 2 * cov) ** .5
        port_stdev = np.sqrt(self.port_weights.T.dot(cov_matrix).dot(self.port_weights))

        # Calculate mean of investment
        self.mean_investment = (1 + port_mean) * self.initial_investment

        # Calculate standard deviation of investmnet
        self.stdev_investment = self.initial_investment * port_stdev
        
    def VaR_Norm(self):
        # Estimating using Normal distribution
        cutoff_norm = norm.ppf(self.conf_level, self.mean_investment, self.stdev_investment)
        
        # Finally, we can calculate the VaR at our confidence interval
        var_norm = self.initial_investment - cutoff_norm
        
        return var_norm
    
    def VaR_T(self):
        # Estimating using t distribution
        cutoff_t = t.ppf(self.conf_level, self.mean_investment, self.stdev_investment)

        var_t = self.initial_investment - cutoff_t
        
        return var_t

In [248]:
var = ValueAtRisk(initial_investment=1000, port_weights=[.2, .2, .5, .1], ret=ret)

In [251]:
print(f"The investment of {initial_investment} is not going lose more than {var.VaR_Norm()} at a confidence level of {(1- conf_level1) * 100}% over {period.days} days (Assuming the return is Normally distributed).")

The investment of 10000 is not going lose more than 39.75290517085841 at a confidence level of 95.0% over 1826 days (Assuming the return is Normally distributed).


In [252]:
print(f"The investment of {initial_investment} is not going lose more than {var.VaR_T()} at a confidence level of {(1- conf_level1) * 100}% over {period.days} days (Assuming the return is t distributed).")

The investment of 10000 is not going lose more than 976.4336361204981 at a confidence level of 95.0% over 1826 days (Assuming the return is t distributed).
